In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [4]:
llm_model = "gpt-3.5-turbo"
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [6]:
# question = """Classes A and B start from the school at the same time to go to a park that is 51 km away. Class A walks at a speed of 4 km/h, and Class B walks at a speed of 5 km/h. The school has a car that can travel at 60 km/h and can carry one class of students at a time. To ensure that both classes arrive at the park in the shortest time possible, how many kilometres do Classes A and B need to walk?"""

# # APE (Automatic Prompt Engineer)
# method = """
# Take a deep breath.
# Let's work this out in a step-by-step way to be sure we have the right answer.
# """

# prompt = f"""Solve the question that is delimited by triple backticks by {method}.
# question: ```{question}```
# """

# prompt_template = ChatPromptTemplate.from_template(prompt)
# solution = prompt_template.format_messages(
#                     method=method,
#                     question=question)

# response = chat(solution)
# print(response.content)

In [14]:
solution = f"""
To solve this question, we need to find the optimal distance that Classes A and B should walk in order to minimize the total time taken for both classes to reach the park.

Let's start by calculating the time it takes for Class A to walk the entire distance of 51 km. We can use the formula: time = distance / speed.

For Class A:
Time taken = 51 km / 4 km/h = 12.75 hours

Now, let's calculate the time it takes for Class B to walk the entire distance of 51 km.
For Class B:
Time taken = 51 km / 5 km/h = 10.2 hours

Since the car can carry only one class at a time, we need to find a way to minimize the total time taken for both classes.

One way to do this is to calculate the time it takes for the car to travel the entire distance of 51 km, drop off one class, and then go back to pick up the other class.

For the car:
Time taken = (51 km / 60 km/h) + (51 km / 60 km/h) = 1.7 hours + 1.7 hours = 3.4 hours

Now, let's calculate the time it takes for Class A to walk a certain distance, while the car carries Class B.

Let's assume Class A walks x km before the car picks up Class B. In this case, Class A will walk the remaining distance of (51 - x) km.

Time taken for Class A = (51 - x) km / 4 km/h = (51 - x) / 4 hours

Since the car takes 3.4 hours to complete the round trip, the time taken for Class B to walk the remaining distance of x km is:

Time taken for Class B = x km / 5 km/h = x / 5 hours

To minimize the total time, we need to find the value of x that minimizes the sum of the times for both classes.

Total time = (51 - x) / 4 + x / 5 + 3.4

To find the value of x that minimizes the total time, we can take the derivative of the total time with respect to x and set it equal to zero.

d(Total time) / dx = -1/4 + 1/5 = 0

Simplifying the equation, we get:

-5/20 + 4/20 = 0

-1/20 = 0

This equation has no solution, which means that the total time is not minimized by having Class A walk a certain distance while the car carries Class B.

Therefore, the optimal solution is for both classes to walk the entire distance of 51 km.

So, Classes A and B need to walk 51 km each.
"""

In [15]:
variable_schema = ResponseSchema(name="variable",
                             description="variable that needs to be sovled by the solution.")

value_schema = ResponseSchema(name="value",
                             description="value of the variable that the solution provides.")

response_schemas = [variable_schema, 
                    value_schema]

In [16]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"variable": string  // variable that needs to be sovled by the solution.
	"value": string  // value of the variable that the solution provides.
}
```


In [17]:
answer_template = """\
For the following solution, extract the following information:

variable: variable that needs to be sovled by the solution \

value: value of the variable that the solution provides \

solution: {solution}

{format_instructions}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template=answer_template)
messages = prompt.format_messages(solution=solution, 
                                format_instructions=format_instructions)

In [19]:
print(messages[0].content)

For the following solution, extract the following information:

variable: variable that needs to be sovled by the solution 
value: value of the variable that the solution provides 
solution: 
To solve this question, we need to find the optimal distance that Classes A and B should walk in order to minimize the total time taken for both classes to reach the park.

Let's start by calculating the time it takes for Class A to walk the entire distance of 51 km. We can use the formula: time = distance / speed.

For Class A:
Time taken = 51 km / 4 km/h = 12.75 hours

Now, let's calculate the time it takes for Class B to walk the entire distance of 51 km.
For Class B:
Time taken = 51 km / 5 km/h = 10.2 hours

Since the car can carry only one class at a time, we need to find a way to minimize the total time taken for both classes.

One way to do this is to calculate the time it takes for the car to travel the entire distance of 51 km, drop off one class, and then go back to pick up the other cla

In [20]:
response = chat(messages)

In [21]:
print(response.content)

```json
{
	"variable": "x",
	"value": "51"
}
```


In [22]:
output_dict = output_parser.parse(response.content)

In [23]:
output_dict

{'variable': 'x', 'value': '51'}

In [25]:
output_dict.get('value')

'51'